<a href="https://colab.research.google.com/github/furkanuyanik/churn-analysis/blob/master/StartedChurnAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

# CSV dosyasını yükle 

In [5]:
telcom = pd.read_csv(r"./churn.csv")
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Null olan değerlerin toplam sayısı

In [6]:
print(telcom.isnull().sum().values.sum())

0


# Değerleri küçük harflere dönüştürme

In [8]:
for column in telcom.columns:
    try:
        telcom[column] = telcom[column].str.lower()
    except:
        print(column, "couldn't convert")

telcom.head()

SeniorCitizen couldn't convert
tenure couldn't convert
MonthlyCharges couldn't convert


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-vhveg,female,0,yes,no,1,no,no phone service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one year,no,mailed check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no phone service,dsl,yes,no,yes,yes,no,no,one year,no,bank transfer (automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber optic,no,no,no,no,no,no,month-to-month,yes,electronic check,70.70,151.65,yes


# Özniteliklerin dataType listesi

In [9]:
for column in telcom.columns:
    print("##", column, ":\n\r" , telcom[column].dtype)

## customerID :
 object
## gender :
 object
## SeniorCitizen :
 int64
## Partner :
 object
## Dependents :
 object
## tenure :
 int64
## PhoneService :
 object
## MultipleLines :
 object
## InternetService :
 object
## OnlineSecurity :
 object
## OnlineBackup :
 object
## DeviceProtection :
 object
## TechSupport :
 object
## StreamingTV :
 object
## StreamingMovies :
 object
## Contract :
 object
## PaperlessBilling :
 object
## PaymentMethod :
 object
## MonthlyCharges :
 float64
## TotalCharges :
 object
## Churn :
 object


# Object olan özniteliklerin dataType'larını değiştiriyoruz.

In [10]:
telcom['TotalCharges'] = telcom['TotalCharges'].replace(r'\s+', np.nan, regex=True)
telcom['TotalCharges'] = pd.to_numeric(telcom['TotalCharges'])

telcom.head() 

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-vhveg,female,0,yes,no,1,no,no phone service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one year,no,mailed check,56.95,1889.50,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no phone service,dsl,yes,no,yes,yes,no,no,one year,no,bank transfer (automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber optic,no,no,no,no,no,no,month-to-month,yes,electronic check,70.70,151.65,yes


# Değerleri Yes/No olan kategorik öznitelikleri sayısal özniteliklere dönüştürme

In [11]:
columns_to_convert = ['Partner', 
                      'Dependents', 
                      'PhoneService', 
                      'PaperlessBilling', 
                      'Churn']

for item in columns_to_convert:
    telcom[item].replace(to_replace='yes', value=1, inplace=True)
    telcom[item].replace(to_replace='no',  value=0, inplace=True)

telcom.head()                      

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-vhveg,female,0,1,0,1,0,no phone service,dsl,no,yes,no,no,no,no,month-to-month,1,electronic check,29.85,29.85,0
1,5575-gnvde,male,0,0,0,34,1,no,dsl,yes,no,yes,no,no,no,one year,0,mailed check,56.95,1889.50,0
2,3668-qpybk,male,0,0,0,2,1,no,dsl,yes,yes,no,no,no,no,month-to-month,1,mailed check,53.85,108.15,1
3,7795-cfocw,male,0,0,0,45,0,no phone service,dsl,yes,no,yes,yes,no,no,one year,0,bank transfer (automatic),42.30,1840.75,0
4,9237-hqitu,female,0,0,0,2,1,no,fiber optic,no,no,no,no,no,no,month-to-month,1,electronic check,70.70,151.65,1


# One-Hot Encoding metodu ile Kategorik değerlerden Sayısal değerlere dönüşüm işlemleri

In [0]:
num_telcom = pd.get_dummies(telcom)
num_telcom.fillna(value=0, inplace=True)

# CustomerID niteliğine ihtiyacımız olmadığı için kaldırıyoruz.

In [13]:
customerId = telcom["customerID"]
del telcom["customerID"]

telcom.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,female,0,1,0,1,0,no phone service,dsl,no,yes,no,no,no,no,month-to-month,1,electronic check,29.85,29.85,0
1,male,0,0,0,34,1,no,dsl,yes,no,yes,no,no,no,one year,0,mailed check,56.95,1889.50,0
2,male,0,0,0,2,1,no,dsl,yes,yes,no,no,no,no,month-to-month,1,mailed check,53.85,108.15,1
3,male,0,0,0,45,0,no phone service,dsl,yes,no,yes,yes,no,no,one year,0,bank transfer (automatic),42.30,1840.75,0
4,female,0,0,0,2,1,no,fiber optic,no,no,no,no,no,no,month-to-month,1,electronic check,70.70,151.65,1


In [14]:
churnerList = telcom[telcom["Churn"] == 1]
nonChurnerList = telcom[telcom["Churn"] == 0]

print(churnerList)
print(nonChurnerList)

      gender  SeniorCitizen  Partner  ...  MonthlyCharges  TotalCharges  Churn
2       male              0        0  ...           53.85        108.15      1
4     female              0        0  ...           70.70        151.65      1
5     female              0        0  ...           99.65        820.50      1
8     female              0        1  ...          104.80       3046.05      1
13      male              0        0  ...          103.70       5036.30      1
...      ...            ...      ...  ...             ...           ...    ...
7021    male              0        0  ...           59.80        727.80      1
7026  female              0        0  ...           44.20        403.35      1
7032    male              1        0  ...           75.75         75.75      1
7034  female              0        0  ...          102.95       6886.25      1
7041    male              1        1  ...           74.40        306.60      1

[1869 rows x 20 columns]
      gender  SeniorCitize

# **Correlation Matrix**

In [0]:
# df = num_telcom.corr()
print(num_telcom)
# matrix_cols = correlation.columns.tolist()

# rs = np.random.RandomState(0)
# corr = df.corr()
# corr.style.background_gradient(cmap='coolwarm')
# 'RdBu_r' & 'BrBG' are other good diverging colormaps